In [1]:
import os
import torch
import polars as pl
import numpy as np
import seaborn as sns
import warnings
from feature_eng.scalers import ranged_scaler
warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
cats_df = pl.read_csv("data/data.csv", separator=",")  

In [3]:
metadata = pl.read_csv('data/metadata.csv',separator=',')


In [4]:
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2023-01-01 00:00:00""",0.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""2023-01-01 00:00:01""",0.0,1.0,20.080031,0.00002,0.0002,0.0,0.0,0.0,0.0,0.0,4.9939e-7,0.000789,0.0,0.0,0.0,0.000021,0.001229,0.0,0.0
"""2023-01-01 00:00:02""",0.0,1.0,20.276562,0.00004,0.0004,0.0,0.0,0.0,0.0,0.0,0.000001,0.003115,0.0,0.0,0.0,0.000104,0.004833,0.0,0.0
"""2023-01-01 00:00:03""",0.0,1.0,20.730938,0.00006,0.0006,0.0,0.0,0.0,0.0,0.0,0.000003,0.006914,0.0,0.0,0.0,0.000285,0.010688,0.0,0.0
"""2023-01-01 00:00:04""",0.0,1.0,21.118101,0.00008,0.0008,0.0,0.0,0.0,0.0,0.0,0.000005,0.012123,0.0,0.0,0.0,0.000601,0.018669,0.0,0.0


In [5]:
metadata.head()


start_time,end_time,root_cause,affected,category
str,str,str,str,i64
"""2023-01-12 15:11:45""","""2023-01-12 15:20:05""","""bso3""","""['cfo1']""",12
"""2023-01-12 16:27:46""","""2023-01-12 17:51:06""","""bso3""","""['cfo1']""",1
"""2023-01-12 18:19:35""","""2023-01-12 18:36:15""","""bfo2""","""['cso1']""",8
"""2023-01-12 20:46:32""","""2023-01-12 20:51:32""","""bed2""","""['ced1']""",7
"""2023-01-13 05:57:10""","""2023-01-13 06:02:10""","""bfo1""","""['cfo1']""",9


In [6]:
for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [7]:
cats_df.head()


timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2023-01-01 00:00:00""",0.0,0.142857,-0.5,-4.1078e-14,2.0428e-14,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180547,-0.507953,-0.716059,-0.774361,0.100389,-0.186623,0.0,0.0
"""2023-01-01 00:00:01""",0.0,0.142857,-0.495998,0.00002,0.0002,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.18054,-0.507953,-0.716059,-0.774361,0.100389,-0.186618,0.0,0.0
"""2023-01-01 00:00:02""",0.0,0.142857,-0.486172,0.00004,0.0004,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180519,-0.507953,-0.716059,-0.774361,0.10039,-0.186604,0.0,0.0
"""2023-01-01 00:00:03""",0.0,0.142857,-0.463453,0.00006,0.0006,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180484,-0.507953,-0.716059,-0.774361,0.100391,-0.18658,0.0,0.0
"""2023-01-01 00:00:04""",0.0,0.142857,-0.444095,0.00008,0.0008,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180437,-0.507953,-0.716059,-0.774361,0.100393,-0.186548,0.0,0.0


In [13]:
from variational_ae.vae import *
from bayesian_inference.causal_inference import bayesian_network_inference

In [15]:
import warnings
warnings.filterwarnings('ignore')
new_metadata = []
iteration = 0
for row in metadata.iter_rows(named=True):
    try:
        start = row['start_time']
        end = row['end_time']
        anomaly = eval(row['affected'])[0]
        root_cause = row['root_cause']
        mod_df = cats_df.filter( (pl.col('timestamp')>= start) & (pl.col('timestamp') <= end))
        cols_to_exclude=['timestamp','y','category']
        mod_df = mod_df.drop(cols_to_exclude)
        data = mod_df.to_numpy()
        vae = VAE(input_dim = data.shape[1],latent_dim = data.shape[1])
        train_vae(vae,data,epochs=50)
        bnn = bayesian_network_inference(anomaly=anomaly,vae_model=vae,model_dataframe=mod_df,input_numpy_data=data)
        bnn.causal_inference()
        if root_cause == bnn.potential_causes[0][0]:
            row['cause_1']=1
        else:
            row['cause_1']=0
        if root_cause == bnn.potential_causes[1][0]:
            row['cause_2']=1
        else:
            row['cause_2']=0
        if root_cause == bnn.potential_causes[2][0]:
            row['cause_3']=1
        else:
            row['cause_3']=0
    
        new_metadata.append(row)
    except MemoryError:
        start = row['start_time']
        end = row['end_time']
        anomaly = eval(row['affected'])[0]
        root_cause = row['root_cause']
        mod_df = cats_df.filter( (pl.col('timestamp')>= start) & (pl.col('timestamp') <= end))
        cols_to_exclude=['timestamp','y','category']
        mod_df = mod_df.drop(cols_to_exclude)
        data = mod_df.to_numpy()
        vae = VAE(input_dim = data.shape[1],latent_dim = data.shape[1])
        train_vae(vae,data,epochs=50)
        bnn = bayesian_network_inference(anomaly=anomaly,vae_model=vae,model_dataframe=mod_df,input_numpy_data=data,scoring_method = 'bicscore')
        bnn.causal_inference()
        try:
            if root_cause == bnn.potential_causes[0][0]:
                row['cause_1']=1
            else:
                row['cause_1']=0
            if root_cause == bnn.potential_causes[1][0]:
                row['cause_2']=1
            else:
                row['cause_2']=0
            if root_cause == bnn.potential_causes[2][0]:
                row['cause_3']=1
            else:
                row['cause_3']=0
        
            new_metadata.append(row)
        except:
            None
    except:
        continue
        

Epoch 1/50, Loss: 5835.1458
Epoch 11/50, Loss: 1704.2271
Epoch 21/50, Loss: 1680.7676
Epoch 31/50, Loss: 1678.8288
Epoch 41/50, Loss: 1678.5359


  0%|          | 16/1000000 [00:04<75:05:59,  3.70it/s]


Epoch 1/50, Loss: 28381.0392
Epoch 11/50, Loss: 17445.9286
Epoch 21/50, Loss: 17372.0657
Epoch 31/50, Loss: 17333.8503
Epoch 41/50, Loss: 17320.6687


  0%|          | 39/1000000 [10:27<4466:59:56, 16.08s/it] 


Epoch 1/50, Loss: 28223.1653
Epoch 11/50, Loss: 17457.4352
Epoch 21/50, Loss: 17384.8352
Epoch 31/50, Loss: 17341.2682
Epoch 41/50, Loss: 17322.3875


...

Epoch 1/50, Loss: 30830.4891
Epoch 11/50, Loss: 18561.5339
Epoch 21/50, Loss: 18568.3202
Epoch 31/50, Loss: 18556.2228
Epoch 41/50, Loss: 18539.2107


  0%|          | 56/1000000 [13:03<3886:53:07, 13.99s/it] 


Epoch 1/50, Loss: 29050.2875
Epoch 11/50, Loss: 18549.3607
Epoch 21/50, Loss: 18550.6399
Epoch 31/50, Loss: 18532.4496
Epoch 41/50, Loss: 18522.1773


  0%|          | 0/1000000 [00:07<?, ?it/s]


MemoryError: Unable to allocate 191. MiB for an array with shape (5001, 5001) and data type float64

In [17]:
len(new_metadata)

79

In [20]:
bnn_stats

start_time,end_time,root_cause,affected,category,cause_1,cause_2,cause_3
str,str,str,str,i64,i64,i64,i64
"""2023-01-12 15:11:45""","""2023-01-12 15:20:05""","""bso3""","""['cfo1']""",12,0,0,0
"""2023-01-12 18:19:35""","""2023-01-12 18:36:15""","""bfo2""","""['cso1']""",8,0,0,0
"""2023-01-12 20:46:32""","""2023-01-12 20:51:32""","""bed2""","""['ced1']""",7,0,0,0
"""2023-01-13 05:57:10""","""2023-01-13 06:02:10""","""bfo1""","""['cfo1']""",9,0,0,0
"""2023-01-13 10:33:15""","""2023-01-13 10:44:55""","""bso1""","""['cfo1']""",4,0,0,0
…,…,…,…,…,…,…,…
"""2023-01-29 15:55:32""","""2023-01-29 16:02:12""","""bfo2""","""['cso1']""",10,0,0,0
"""2023-01-29 23:45:43""","""2023-01-29 23:57:23""","""bed2""","""['ced1']""",8,0,0,0
"""2023-01-30 03:20:25""","""2023-01-30 03:32:05""","""bed2""","""['ced1']""",12,0,0,0


In [19]:
bnn_stats = pl.DataFrame(new_metadata)
agg_stats = bnn_stats.select(pl.sum("cause_1", "cause_2",'cause_3'))
agg_stats.select(pl.sum_horizontal(pl.all())).item()/bnn_stats.shape[0]

0.0

In [11]:
bnn.potential_causes[0][0]

'b'

In [ ]:
cols_to_exclude=['timestamp','y','category']
mod_df = mod_df.drop(cols_to_exclude)
data = mod_df.to_numpy()

In [ ]:
vae = VAE(input_dim = data.shape[1],latent_dim = data.shape[1])
train_vae(vae,data,epochs=50)

In [ ]:
data.shape[1]

In [ ]:
class bayesian_network_inference:
    def __init__(self,anomaly,vae_model,model_dataframe,input_numpy_data, scoring_method = 'bdeuscore'):
        self.anomaly = anomaly
        self.vae_model = vae_model
        self.mod_df = model_dataframe
        self.column_names = self.mod_df.columns
        self.input_numpy_data = input_numpy_data
        self.scoring_method=scoring_method
        self.latent_df = None
        self.bayesian_model = None
        self.rca_scores = None
        self.potential_causes = []

    def _get_latent_variables(self):
        self.vae_model.eval()
        with torch.no_grad():
            data_tensor = torch.tensor(self.input_numpy_data, dtype=torch.float32)
            mu, _ = self.vae_model.encode(data_tensor)
            return mu.numpy()

    def _prep_latent_data(self):
        latent_data = get_latent_variables(vae, data)
        self.latent_df = pd.DataFrame(latent_data, columns=[f"z{i}" for i in range(latent_data.shape[1])])
        self.latent_df.columns = self.mod_df.columns

    def _train_bayes(self):
        # Train Bayesian Network
        hc = HillClimbSearch(self.latent_df)
        best_model = hc.estimate(scoring_method=self.scoring_method)
        self.bayesian_model = BayesianNetwork(best_model.edges())
        self.bayesian_model.fit(self.latent_df, estimator=MaximumLikelihoodEstimator)

    def draw_graph(self,name):
        viz = bayesian_model.to_graphviz()
        viz.draw( name + '.png', prog='neato')
        Image(name + '.png')

    def causal_inference(self,n_causes=3):
        self._get_latent_variables()
        self._prep_latent_data()
        self._train_bayes()

        inference = CausalInference(self.bayesian_model)
        scores = []
        columns = self.latent_df.columns
        test_columns = []
        for col in columns:
            if col != self.anomaly:
                try:
                    inference.get_all_backdoor_adjustment_sets(X=col, Y=self.anomaly)
                except:
                    None
                try:
                    inference.get_all_frontdoor_adjustment_sets(X=col, Y=self.anomaly)
                except:
                    None
                test_columns.append(col)
                scores.append(inference.estimate_ate(X=col, Y=self.anomaly, data=self.latent_df, estimator_type="linear"))

        rca_indices = sorted(range(len(scores)), key=lambda i: scores[i])[-n_causes:]
        self.rca_scores = dict(zip(test_columns,scores))
        for index in rca_indices:
            self.potential_causes.append( (test_columns[index]))
        self.potential_causes.sort(key=lambda tup: tup[1],reverse=True)


 
        

In [ ]:
bnn = bayesian_network_inference(anomaly=anomaly,vae_model=vae,model_dataframe=mod_df,input_numpy_data=data)
bnn.causal_inference()

In [ ]:
causes = bnn.potential_causes

In [ ]:
causes.sort(key=lambda tup: tup[1],reverse=True)

In [ ]:
causes

In [ ]:
bnn.draw_graph(name='mod1')

In [ ]:
# Define Bayesian Network
def get_latent_variables(model, data):
    model.eval()
    with torch.no_grad():
        data_tensor = torch.tensor(data, dtype=torch.float32)
        mu, _ = model.encode(data_tensor)
        return mu.numpy()


In [ ]:
def train_bayes(latent_df,scoring_method = 'bdeuscore'):
    # Train Bayesian Network
    hc = HillClimbSearch(latent_df)
    best_model = hc.estimate(scoring_method='bdeuscore')
    bayesian_model = BayesianNetwork(best_model.edges())
    bayesian_model.fit(latent_df, estimator=MaximumLikelihoodEstimator)
    return bayesian_model

In [ ]:
# Prepare latent data
latent_data = get_latent_variables(vae, data)
latent_df = pd.DataFrame(latent_data, columns=[f"z{i}" for i in range(latent_data.shape[1])])
latent_df.columns = mod_df.columns

In [ ]:
latent_df.shape

In [ ]:
# Train Bayesian Network
hc = HillClimbSearch(latent_df)
best_model = hc.estimate(scoring_method='bdeuscore')
bayesian_model = BayesianNetwork(best_model.edges())
bayesian_model.fit(latent_df, estimator=MaximumLikelihoodEstimator)


In [ ]:
from IPython.display import Image

viz = bayesian_model.to_graphviz()
viz.draw('test.png', prog='neato')
Image('test.png')

In [ ]:
from pgmpy.inference.CausalInference import CausalInference
inference = CausalInference(bayesian_model)
inference.get_all_backdoor_adjustment_sets(X="bso3", Y=anomaly)
inference.get_all_frontdoor_adjustment_sets(X="bso3", Y=anomaly)

In [ ]:
scores = []
columns = latent_df.columns
test_columns = []
for col in latent_df.columns:
    if col != anomaly:
        inference.get_all_backdoor_adjustment_sets(X=col, Y=anomaly)
        inference.get_all_frontdoor_adjustment_sets(X=col, Y=anomaly)
        test_columns.append(col)
        scores.append(inference.estimate_ate(X=col, Y=anomaly, data=latent_df, estimator_type="linear"))

In [ ]:
rca_indices = sorted(range(len(scores)), key=lambda i: scores[i])[-3:]

In [ ]:
potential_causes = []
for index in rca_indices:
    potential_causes.append( (test_columns[index],scores[index]))


In [ ]:
potential_causes

In [ ]:
test_columns[13]

In [ ]:
test_columns[12]

In [ ]:
test_columns[1]

In [ ]:
metadata